Demo scraper de URLS


Importar librerias necesarias


In [1]:
from bs4 import BeautifulSoup     #Web scraper lib
import requests                   #Url Requester lib
from datetime import datetime     #Todays date lib
import json                       #Json converter lib
from urllib.parse import urlparse #Get Domain from URLS
import json                       #Json Responses
from datetime import datetime     #TimeStamps   

Ay no! 
Te faltaron librerias?


In [1]:
!pip install -r requirements.txt  #Aqui instalas dependencias faltantes


[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Variables para control de multiple url o simple


In [ ]:
URL = 'https://www.eldiariodechihuahua.mx/local/judicializadas-2675-denuncias-en-caso-aras-20230706-2073545.html'

In [ ]:
URL = ['https://www.tiempo.com.mx/noticia/threads_record_10_millones_usuarios_7_horas_lanzamiento_red_social/', 'https://www.eldiariodechihuahua.mx/local/judicializadas-2675-denuncias-en-caso-aras-20230706-2073545.html']

In [ ]:
URL = ['https://stackoverflow.com/questions/58224512/how-can-i-reset-and-re-run-this-code-every-2-minutes-in-python-the-while-loop-f','https://www.uv.es/jac/guia/link1.htm', 'https://laparadadigital.com/']

In [2]:
URL = ['https://laparadadigital.com/', 'https://www.tiempo.com.mx/noticia/threads_record_10_millones_usuarios_7_horas_lanzamiento_red_social/', 'https://www.eldiariodechihuahua.mx/local/judicializadas-2675-denuncias-en-caso-aras-20230706-2073545.html']

In [3]:
TimeNow = datetime.now()


Funcion para definir si es multiple url o una simple url.
Convertir si es un STR a LIST (array)



In [4]:
def MultiUrlIdentifier(URL):
    UrlArray = []
    if type(URL) is (list):
        UrlArray = URL
    elif type(URL) is (str):
        UrlArray = [URL]

    return UrlArray
MultiUrlIdentifier(URL)

['https://laparadadigital.com/',
 'https://www.tiempo.com.mx/noticia/threads_record_10_millones_usuarios_7_horas_lanzamiento_red_social/',
 'https://www.eldiariodechihuahua.mx/local/judicializadas-2675-denuncias-en-caso-aras-20230706-2073545.html']

Ahora que homologamos las entradas de url necesitaremos buscar por cada url si tiene mas vertientes.
Para eso cada URL se tiene que identificar su dominio.


In [5]:
def DomainIdentifier(URL):
    UrlDomain = []
    UrlArray = MultiUrlIdentifier(URL)
    for Url in UrlArray:
        domain  = urlparse(Url,scheme='',allow_fragments=False)
        UrlDomain.append(["https://" + domain.netloc])
    return UrlDomain
DomainIdentifier(URL)

[['https://laparadadigital.com'],
 ['https://www.tiempo.com.mx'],
 ['https://www.eldiariodechihuahua.mx']]

Ya identificamos los dominios. Ahora necesitamos revisar por cada Url insertada, todas sus posibles vertientes.  
Para ello haremos una lista por cada url existente

In [6]:
def SlashAssigner(URL):             #Al final de cada Href sin Dominio necesita un Slash 
    if not URL.startswith("/"):
        return "/" + URL
    else:
        return URL

Necesitaremos obtener los links enteros y a mitad (Esto depende de la pagina, como la contruyeron, etc.) de cada HREF

In [7]:
def AHalfTagFinder(URL,DomainTrue , DomainURLS):                #Aun no se habilita la version con POWER , para eso exite DomainURLS
    Domain_List = DomainIdentifier(URL)                         #Obtenemos nuestros URL Dominio
    Returner = []       
    loop_counter = 0
    for U in Domain_List:                                       #Por cada URL Dominio , obtener una Sopa
        Returner.append([loop_counter])
        Url_Smart_List = []
        soup = None
        Page = requests.get(U[0])
        soup = BeautifulSoup(Page.text, 'html.parser')
        for link in soup.find_all('a'):                         #Por cada Sopa obtener todos los links
            Half_Link = str(link.get('href'))
            Half_Link = SlashAssigner(Half_Link)
            Full_Link = ''
            if 'http' not in Half_Link :                        #asegurar que el link sea a la mitad
                Full_Link = U[0] + Half_Link
                if Full_Link not in Url_Smart_List:             #concatenar si no existe
                    Url_Smart_List.append(str(Full_Link))
        Returner[loop_counter].append(Url_Smart_List)
        loop_counter+=1
    return Returner
# AHalfTagFinder(URL,False,"")

Repetiremos la version anterior pero para obtener los links completos y del mismo dominio

In [8]:
def AFullTagFinder(URL,DomainTrue , DomainURLS): 
    Domain_List = DomainIdentifier(URL)
    Returner = []
    loop_counter = 0
    for U in Domain_List:
        Returner.append([loop_counter])
        Url_Smart_List = []
        soup = None
        Page = requests.get(U[0])
        soup = BeautifulSoup(Page.text, 'html.parser')
        for link in soup.find_all('a'):
            Half_Link = str(link.get('href'))
            if 'http'  in Half_Link and U[0] in Half_Link:  #Ignoramos que sea un link completo  y que el dominio se encuentre en el link
                Full_Link = Half_Link
                if Full_Link not in Url_Smart_List: 
                    Url_Smart_List.append(str(Full_Link))
        Returner[loop_counter].append(Url_Smart_List)
        loop_counter+=1
    return Returner
# AFullTagFinder(URL,False,"")

Sumamos los dos valores Enteros y Mitades

In [9]:
def AppendoListo(List1, List2):
    sumA = {}
    for i in range(len(List1)):
        NewList = List1[i][1] + List2[i][1]
        sumA[i] = Enlistator(NewList)
      
    return sumA
# AppendoListo(a,b)

Por ser un listado , se puede eliminar el duplicado, al momento de ecerrarlo en brackets dejara de ser listado y tendriamos que iterar, por lo que mejor eliminamos duplicados y al final enlistamos los link para poder concatenar su respectivo body

In [10]:
def Enlistator(List):
    NewList = []
    for l in List:
        NewList.append([l])
    print(NewList)
    
    return NewList
# Enlistator(newrls)

Tags para probar

In [11]:
    a = AHalfTagFinder(URL,"False","")
    b = AFullTagFinder(URL,"False","")

In [12]:
def HalfAndFullConcatenator(URL,DomainTrue,DomainUrl):
    a = AHalfTagFinder(URL,DomainTrue,DomainUrl)
    b = AFullTagFinder(URL,DomainTrue,DomainUrl)
    c = AppendoListo(a,b)

    return c
    
# HalfAndFullConcatenator(URL,False,"")

In [ ]:
Extraccion de body per link

In [13]:
def BodyReader(Links,TimeNow):
    for MainUrl in Links :
        print(MainUrl)
        for i in range(len(Links[MainUrl])):
            page = requests.get(Links[MainUrl][i][0])
            soup = BeautifulSoup(page.content , 'lxml')
            soupon = str(soup.prettify().encode('utf-8'))  
            JsonData = str(TimeNow)  + str(json.dumps(soupon))
            Links[MainUrl][i].append(JsonData)
            
    return Links

# BodyReader(flavotta,TimeNow)

Demostracion de Body y resultado final

In [14]:
flavotta = HalfAndFullConcatenator(URL,False,"")
mainDish = BodyReader(flavotta,TimeNow)

[['https://laparadadigital.com/#'], ['https://laparadadigital.com/None'], ['https://laparadadigital.com/#recent-video-tabs-206241'], ['https://laparadadigital.com/#recent-video-tabs-206164'], ['https://laparadadigital.com/#recent-video-tabs-135619'], ['https://laparadadigital.com/#recent-video-tabs-73559'], ['https://laparadadigital.com/#recent-video-tabs-72714'], ['https://laparadadigital.com/cdn-cgi/l/email-protection'], ['https://laparadadigital.com/'], ['https://laparadadigital.com/category/destacadas/'], ['https://laparadadigital.com/category/cd-juarez/'], ['https://laparadadigital.com/category/lengua-viperina/'], ['https://laparadadigital.com/category/nacional/'], ['https://laparadadigital.com/category/global/'], ['https://laparadadigital.com/category/cancha/'], ['https://laparadadigital.com/category/monero/'], ['https://laparadadigital.com/category/cultura-jet-set/'], ['https://laparadadigital.com/category/ladies/'], ['https://laparadadigital.com/category/relax/'], ['https://lap

In [27]:
mainDish[0][0][1]

'2023-07-18 12:36:48.445390"b\'<!DOCTYPE html>\\\\n<!--[if lt IE 7]> <html class=\\"no-js ie6 oldie\\" lang=\\"es\\"> <![endif]-->\\\\n<!--[if IE 7]>    <html class=\\"no-js ie7 oldie\\" lang=\\"es\\"> <![endif]-->\\\\n<!--[if IE 8]>    <html class=\\"no-js ie8 oldie\\" lang=\\"es\\"> <![endif]-->\\\\n<!--[if IE 9]>    <html class=\\"no-js ie9 oldie\\" lang=\\"es\\"> <![endif]-->\\\\n<!--[if gt IE 8]><!-->\\\\n<html class=\\"\\" lang=\\"es-ES\\">\\\\n <!--<![endif]-->\\\\n <head>\\\\n  <meta charset=\\"utf-8\\"/>\\\\n  <title>\\\\n   Tiempo La Noticia Digital | Noticias de Chihuahua\\\\n  </title>\\\\n  <meta content=\\"Noticias, editorial, minuto a minuto, articulos, tiempo, reportajes, videos, galerias, impacto\\" name=\\"keywords\\"/>\\\\n  <meta content=\\"Tiempo - La Noticia Digital.\\" name=\\"description\\"/>\\\\n  <meta content=\\"Tiempo - La Noticia Digital.\\" name=\\"author\\"/>\\\\n  <meta content=\\"z326m52t4qkc8dzi5qlvv9jrtjqg0x\\" name=\\"facebook-domain-verification\\"/